In [ ]:
#! pip install datasets evaluate transformers rouge-score nltk

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# !apt install git-lfs

In [ ]:
import transformers

print(transformers.__version__)

4.40.0


In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

In [ ]:
model_checkpoint = "t5-small"

In [ ]:
pip install transformers datasets evaluate

In [ ]:
from datasets import load_dataset, DatasetDict

# Load the dataset
raw_dataset = load_dataset("samaxr/code-summary-java")
train_data = raw_dataset["train"]
validation_data = raw_dataset["validation"]
test_data = raw_dataset["test"]

# Select a subset of the dataset
subset_size = 500
train_data = train_data.select(range(subset_size))
validation_data = validation_data.select(range(subset_size))
test_data = test_data.select(range(subset_size))

# Create a DatasetDict containing train, validation, and test splits
raw_datasets = DatasetDict({
    "train": train_data,
    "validation": validation_data,
    "test": test_data
})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['code', 'summary'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['code', 'summary'],
        num_rows: 500
    })
    test: Dataset({
        features: ['code', 'summary'],
        num_rows: 500
    })
})

In [ ]:
raw_datasets["train"][0]

{'code': 'protected final void bindIndexed(ConfigurationPropertyName name, Bindable<?> target,\n\t\t\tAggregateElementBinder elementBinder, ResolvableType aggregateType,\n\t\t\tResolvableType elementType, IndexedCollectionSupplier result) {\n\t\tfor (ConfigurationPropertySource source : getContext().getSources()) {\n\t\t\tbindIndexed(source, name, target, elementBinder, result, aggregateType,\n\t\t\t\t\telementType);\n\t\t\tif (result.wasSupplied() && result.get() != null) {\n\t\t\t\treturn;\n\t\t\t}\n\t\t}\n\t}',
 'summary': 'Binds the indexed elements of the configuration property.'}

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,code,summary
0,"private String getEntityManagerFactoryName(String beanName) {\n\t\tif (beanName.length() > ENTITY_MANAGER_FACTORY_SUFFIX.length() && StringUtils\n\t\t\t\t.endsWithIgnoreCase(beanName, ENTITY_MANAGER_FACTORY_SUFFIX)) {\n\t\t\treturn beanName.substring(0,\n\t\t\t\t\tbeanName.length() - ENTITY_MANAGER_FACTORY_SUFFIX.length());\n\t\t}\n\t\treturn beanName;\n\t}",Gets the entity manager factory name.
1,"public void setServletRegistrationBeans(\n\t\t\tCollection<? extends ServletRegistrationBean<?>> servletRegistrationBeans) {\n\t\tAssert.notNull(servletRegistrationBeans,\n\t\t\t\t""ServletRegistrationBeans must not be null"");\n\t\tthis.servletRegistrationBeans = new LinkedHashSet<>(servletRegistrationBeans);\n\t}",Sets the servlet registration beans.
2,"protected Set<String> getExclusions(AnnotationMetadata metadata,\n\t\t\tAnnotationAttributes attributes) {\n\t\tSet<String> excluded = new LinkedHashSet<>();\n\t\texcluded.addAll(asList(attributes, ""exclude""));\n\t\texcluded.addAll(Arrays.asList(attributes.getStringArray(""excludeName"")));\n\t\texcluded.addAll(getExcludeAutoConfigurationsProperty());\n\t\treturn excluded;\n\t}",Returns the set of excluded annotations.
3,"public static int compare(Long[] arrayA, Long[] arrayB)\n {\n int len1 = arrayA.length;\n int len2 = arrayB.length;\n int lim = Math.min(len1, len2);\n\n int k = 0;\n while (k < lim)\n {\n Long c1 = arrayA[k];\n Long c2 = arrayB[k];\n if (!c1.equals(c2))\n {\n return c1.compareTo(c2);\n }\n ++k;\n }\n return len1 - len2;\n }",Compare two Long arrays.
4,public String nextString()\n {\n char[] buffer = new char[nextInt()];\n for (int i = 0; i < buffer.length; ++i)\n {\n buffer[i] = nextChar();\n }\n return new String(buffer);\n },Get the next string in the sequence.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
tokenizer("public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n}")

{'input_ids': [452, 853, 5140, 3, 2, 452, 14491, 3, 12186, 711, 599, 11500, 53, 6306, 908, 3, 8240, 7, 61, 3, 2, 16, 17, 3, 5525, 536, 3274, 305, 117, 16, 17, 3, 5525, 357, 3274, 335, 117, 16, 17, 4505, 3274, 3, 5525, 536, 1768, 3, 5525, 357, 117, 2149, 5, 670, 5, 7260, 40, 29, 599, 121, 134, 440, 10, 96, 1768, 4505, 3670, 3, 2, 3, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_code = tokenizer(["public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n", "class Test{   public static void main(String []args) {  System.out.println(\"My First Java Program.\");  }};"])

print(tokenized_code)

{'input_ids': [[452, 853, 5140, 3, 2, 452, 14491, 3, 12186, 711, 599, 11500, 53, 6306, 908, 3, 8240, 7, 61, 3, 2, 16, 17, 3, 5525, 536, 3274, 305, 117, 16, 17, 3, 5525, 357, 3274, 335, 117, 16, 17, 4505, 3274, 3, 5525, 536, 1768, 3, 5525, 357, 117, 2149, 5, 670, 5, 7260, 40, 29, 599, 121, 134, 440, 10, 96, 1768, 4505, 3670, 3, 2, 1], [853, 2300, 2, 452, 14491, 3, 12186, 711, 599, 11500, 53, 784, 908, 8240, 7, 61, 3, 2, 2149, 5, 670, 5, 7260, 40, 29, 599, 121, 7008, 1485, 10318, 2350, 535, 3670, 3, 2, 117, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
print(tokenizer(text_target=["public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n", "class Test{   public static void main(String []args) {  System.out.println(\"My First Java Program.\");  }};"]))

{'input_ids': [[452, 853, 5140, 3, 2, 452, 14491, 3, 12186, 711, 599, 11500, 53, 6306, 908, 3, 8240, 7, 61, 3, 2, 16, 17, 3, 5525, 536, 3274, 305, 117, 16, 17, 3, 5525, 357, 3274, 335, 117, 16, 17, 4505, 3274, 3, 5525, 536, 1768, 3, 5525, 357, 117, 2149, 5, 670, 5, 7260, 40, 29, 599, 121, 134, 440, 10, 96, 1768, 4505, 3670, 3, 2, 1], [853, 2300, 2, 452, 14491, 3, 12186, 711, 599, 11500, 53, 784, 908, 8240, 7, 61, 3, 2, 2149, 5, 670, 5, 7260, 40, 29, 599, 121, 7008, 1485, 10318, 2350, 535, 3670, 3, 2, 117, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["code"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[21603, 10, 5046, 804, 3, 12186, 3, 8610, 26267, 226, 15, 26, 599, 4302, 26703, 3174, 883, 17, 63, 23954, 564, 6, 7617, 26, 179, 2, 58, 3155, 2387, 6, 4821, 18301, 342, 427, 3335, 279, 77, 588, 3282, 279, 77, 588, 6, 419, 6065, 179, 25160, 12955, 25160, 6, 419, 6065, 179, 25160, 3282, 25160, 6, 11507, 15, 26, 9939, 12252, 134, 413, 5900, 49, 741, 61, 3, 2, 21, 41, 4302, 26703, 3174, 883, 17, 63, 23799, 1391, 3, 10, 129, 4302, 6327, 9960, 5, 2782, 23799, 7, 9960, 61, 3, 2, 3, 8610, 26267, 226, 15, 26, 599, 7928, 6, 564, 6, 2387, 6, 3282, 279, 77, 588, 6, 741, 6, 12955, 25160, 6, 3282, 25160, 3670, 3, 99, 41, 60, 7, 83, 17, 5, 9491, 134, 413, 102, 15310, 9960, 3, 184, 184, 741, 5, 2782, 9960, 3, 55, 2423, 206, 195, 61, 3, 2, 1205, 117, 3, 2, 3, 2, 3, 2, 1], [21603, 10, 452, 3, 12186, 356, 17598, 1655, 18752, 257, 2703, 3247, 599, 6767, 2, 58, 4285, 7, 3, 17598, 1655, 18752, 257, 2703, 152, 2, 58, 3155, 3155, 3, 3473, 1655, 18752, 257, 2703, 3247, 61, 3, 2, 282, 7, 49, 17, 

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
pip install transformers[torch]


In [ ]:
pip install accelerate -U

In [ ]:
batch_size = 16

model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
 '''
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
'''
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    prediction_loss_only=True,  # As we are doing regression (predicting the token ids), only compute the loss
    logging_dir="./logs",
)

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,  # Your pre-trained sequence-to-sequence model
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,  # Your function to compute metrics
)

# Start training
trainer.train()


Step,Training Loss


TrainOutput(global_step=375, training_loss=2.5233050130208334, metrics={'train_runtime': 2104.6042, 'train_samples_per_second': 0.713, 'train_steps_per_second': 0.178, 'total_flos': 124217339805696.0, 'train_loss': 2.5233050130208334, 'epoch': 3.0})

In [ ]:
# prompt: saving a model for t5 in collab after that training is done

trainer.save_model('./results')

In [ ]:
# To load the saved model for inference
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch



In [ ]:
# Load the trained model
model_name = BertForSequenceClassification.from_pretrained("./results")

You are using a model of type t5 to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./results and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermedi

In [ ]:
from transformers import BartForConditionalGeneration


In [ ]:
model_name = BartForConditionalGeneration.from_pretrained("./results")

You are using a model of type t5 to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.
Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at ./results and are newly initialized: ['decoder.embed_positions.weight', 'decoder.layernorm_embedding.bias', 'decoder.layernorm_embedding.weight', 'decoder.layers.0.encoder_attn.k_proj.bias', 'decoder.layers.0.encoder_attn.k_proj.weight', 'decoder.layers.0.encoder_attn.out_proj.bias', 'decoder.layers.0.encoder_attn.out_proj.weight', 'decoder.layers.0.encoder_attn.q_proj.bias', 'decoder.layers.0.encoder_attn.q_proj.weight', 'decoder.layers.0.encoder_attn.v_proj.bias', 'decoder.layers.0.encoder_attn.v_proj.weight', 'decoder.layers.0.encoder_attn_layer_norm.bias', 'decoder.layers.0.encoder_attn_layer_norm.weight', 'decoder.layers.0.fc1.bias', 'decoder.layers.0.fc1.weight', 'decoder.layers.0.fc2.bias', 'decoder.layers.0.fc2.weight', 'decoder.layers.0.fi

In [ ]:
model_name.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
input_text = "public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n}"
input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# 5. Generate Output
output_ids = model.generate(input_ids["input_ids"])

# Decode the generated output
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Summary:", output_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1256: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Summary: 


In [ ]:
torch.save(trainer.state_dict(), './result1' )

AttributeError: 'Seq2SeqTrainer' object has no attribute 'state_dict'

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# You can of course substitute your own username and model here if you've trained and uploaded it!
model_name = '/content/results/config.json'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

OSError: Incorrect path_or_model_id: '/content/results/config.json'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
pip show torch

Name: torch
Version: 2.2.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, torchaudio, torchdata, torchtext, torchvision


In [ ]:
'''
code = 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we\'re neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was badly hit by problems, sparking calls to introduce more defences in the area.\nScottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.\nThe Labour Party\'s deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.\nHe said it was important to get the flood protection plan right but backed calls to speed up the process.\n"I was quite taken aback by the amount of damage that has been done," he said.\n"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."\nHe said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.\nHave you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.'
if 't5' in model_name:
    code = "summarize: " + code
tokenized = tokenizer([code], return_tensors='np')
out = model.generate(**tokenized, max_length=128)

# Truncate the input text to fit within the maximum sequence length
max_seq_length = 512
code = code[:max_seq_length]
'''
max_seq_length = 512
#code = code[:max_seq_length]
code='public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n}'

# Tokenize the truncated input
if 't5' in model_name:
    code = "summarize: " + code

# Tokenize the truncated input
tokenized = tokenizer([code], return_tensors='pt')

# Generate output
out = model.generate(**tokenized, max_length=128)

# Decode the generated output
generated_summary = tokenizer.decode(out[0], skip_special_tokens=True)
print(generated_summary)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><unk>


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb#scrollTo=imY1oC3SIrJf

In [ ]:
https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization-tf.ipynb#scrollTo=GbEgJVr_kC6F